### Setup

In [27]:
# llamma index related
!pip install llama-index-core llama-parse llama-index-readers-file python-dotenv

In [65]:
from llama_index.core import Document, SummaryIndex
from llama_index.core.schema import TextNode

import numpy as np
import random 
import torch
import json

## Data Ingestion

### Directory Reader from llamaindex

In [33]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_dir="issuenote",
    recursive=True
)
documents = reader.load_data()
for doc in documents:
    print(doc.metadata)
    print(doc.text)

{'page_label': '1', 'file_name': 'issue(2024-29).pdf', 'file_path': '/home/geonheekim/kimgeonhee/github/LLM-techstack-practice/issuenote/issue(2024-29).pdf', 'file_type': 'application/pdf', 'file_size': 1776434, 'creation_date': '2024-10-30', 'last_modified_date': '2024-10-30'}
1
토큰증권(Security Token Offering)을 통한 녹색채권 발행 사례 및 시사점1    탄소중립 목표 달성을 위해서는 저탄소·친환경 분야에 막대한 자금이 소요될 것으로 예상되나, 
동 자금을 조달하기 위한 국내 녹색금융 시장의 성장세는 더딘 상황이다. 본 고는 우리나라의 녹
색금융 활성화 방안으로 홍콩 등 해외 토큰형 녹색채권의 발행 사례를 살펴보고 국내 활용 가능성 
등을 점검하였다.
2    녹색채권은 친환경 프로젝트에 소요되는 자금을 조달하기 위해 발행되는 채권으로 자금의 사용처
와 환경개선 효과에 대한 투명성이 매우 중요하다. 이에 따라 일반채권에 비해 엄격한 사전·사
후 보고 및 외부 검증 절차를 두고 있다. 그러나 이처럼 복잡한 발행·보고 절차와 발행자와 투자
자 간 구조적인 정보의 비대칭성(그린워싱 우려) 등은 녹색채권 시장 활성화의 주된 제약요인으로 
꼽힌다.
3    녹색채권을 블록체인을 기반으로 하는 토큰증권으로 발행하는 경우 조달자금의 사용처 및 환경영
향에 대한 투명성 제고, 발행 및 사후보고 절차의 간소화, 투자자 기반 확대 등 다양한 효과가 기
대된다. 이를 통해 국내기업, 특히 녹색금융 접근성이 낮았던 중소·중견기업에도 친환경 자금 
조달 기회가 확대될 것으로 기대된다.
4    해외 사례를 보면 홍콩에서는 당국 주도로 녹색국채를 토큰증권 형태로 시범 발행하여 규제 관련 
불확실성을 해소하는 한편 스마트 계약 등을 통한 발행절차의

In [35]:
idx = random.randint(0, len(documents))
print(documents[idx].metadata)
print(documents[idx].text)

{'page_label': '6', 'file_name': 'issue(2024-29).pdf', 'file_path': '/home/geonheekim/kimgeonhee/github/LLM-techstack-practice/issuenote/issue(2024-29).pdf', 'file_type': 'application/pdf', 'file_size': 1776434, 'creation_date': '2024-10-30', 'last_modified_date': '2024-10-30'}
제 2024-29호6
한국은행등 2세대 블록체인이 등장하면서 다양한 권리
를 블록체인에 담으려는 시도가 나타났다.15) 
이와 같은 배경에서 토큰증권이 등장하였으며 주식, 채권의 발행 및 거래에 블록체인 기술을 활용하는 경우 해당 증권의 권리, 거래정보 등은 기존처럼 단일 기관에 집중된 장부가 아닌 다수의 네트워크 참여자들이 관리하는 분산원장에 기록된다.(<그림 6>)
주식
실물증권채권
전자증권 토큰증권
(발행형태)(증권)
분산원장
기재중앙집중식
계좌부 기재실물증서
기재<그림 6> 토큰증권 개념
자료: 금융위원회(저자 재구성)
토큰증권이 발행되는 블록체인 플랫폼은 크
게 누구나 참여 가능한 개방형(public) 블록체인과 사전에 허가받은 대상들만 참여 가능한 허가형(permissioned) 블록체인으로 구분
16)된
다. 이 중 개방형 블록체인은 가장 탈중앙화되어 있는 형태로 모든 참여자가 네트워크에 대한 권한을 공유하고 거래기록이 모두에게 투명하게 공개된다. 이같은 특성을 활용한 탈중앙화 금융(Defi)
17) 등의 분야가 크게 성장하면서,  
개방형 블록체인을 기반으로 다양한 금융생태계가 형성되어 있다. 반면 허가형 블록체인의 경우 네트워크 참여자를 제한하여 부분적으로 분산된 형태를 보이므로 상대적으로 거래처리 속도가 빠르고, 거래정보 보호 수준이 높으며, 규제체계에 특화된 설계가 용이하다는 특성을 지니고 있다. 이에 따라 허가형 플랫폼은 거래 안정성과 규제 준수가 

### Parsing Documents into the Nodes

Splitter

In [49]:
# token text splitter
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter

splitter = SentenceSplitter(
    chunk_size=100,
    chunk_overlap=0,
)
nodes = splitter.get_nodes_from_documents(documents)


In [53]:
# token text splitter
splitter = TokenTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separator=" ",
    backup_separators=[".", "!", "?"]
)
nodes = splitter.get_nodes_from_documents(documents)


In [54]:
print(nodes[3], nodes[3].text)

Node ID: 8ae88309-f17d-409e-b9af-a431eadd5fdc
Text: 활성화 방안으로 홍콩 등 해외 토큰형 녹색채권의 발행 사례를 살펴보고 국내 활성화 방안으로 홍콩 등 해외 토큰형 녹색채권의 발행 사례를 살펴보고 국내


In [55]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'issue(2024-29).pdf',
 'file_path': '/home/geonheekim/kimgeonhee/github/LLM-techstack-practice/issuenote/issue(2024-29).pdf',
 'file_type': 'application/pdf',
 'file_size': 1776434,
 'creation_date': '2024-10-30',
 'last_modified_date': '2024-10-30'}

## Operation

### Querying

In [63]:
from clova_api.completion_executor import ChatCompletionExecutor
from clova_api.sliding_window_executor import SlidingWindowExecutor

In [ ]:
# 스트리밍 응답에서 content 부분만 추출
def parse_stream_response(response):
    content_parts = []
    for line in response.splitlines():
        if line.startswith('data:'):
            data = json.loads(line[5:])
            if 'message' in data and 'content' in data['message']:
                content_parts.append(data['message']['content'])
    content = content_parts[-1] if content_parts else ""
    return content.strip()
 
# 논스트리밍 응답에서 content 부분만 추출
def parse_non_stream_response(response):
    result = response.get('result', {})
    message = result.get('message', {})
    content = message.get('content', '')
    return content.strip()
 
def main():
    # 초기 시스템 프롬프트 설정
    system_prompt = "- HyperCLOVA X는 네이버 클라우드의 하이퍼스케일 AI입니다."
    messages = []
 
    sliding_window_executor = SlidingWindowExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key = '<api_key>',
        api_key_primary_val = '<api_key_primary_val>',
        request_id = '<request_id>'
    )
 
    completion_executor = ChatCompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key = '<api_key>',
        api_key_primary_val = '<api_key_primary_val>',
        request_id = '<request_id>'
    )
 
    # stream 옵션에 따라 응답을 토큰 단위(stream=True), 전체(stream=False)로 받을 수 있습니다.
    stream = True
 
    while True:
        user_input = input("USER: ('exit'으로 종료): ")
        if user_input.lower() in ['exit', 'quit']:
            break
 
        messages.append({"role": "user", "content": user_input})
 
        request_data = {
            "messages": [{"role": "system", "content": system_prompt}] + messages,
            "maxTokens": 100  # 슬라이딩 윈도우에서 사용할 토큰 수
        }
 
        # SlidingWindowExecutor를 사용하여 조정된 메시지 가져오기
        try:
            adjusted_messages = sliding_window_executor.execute(request_data)
            if adjusted_messages == 'Error':
                print("Error adjusting messages with SlidingWindowExecutor")
                continue
        except Exception as e:
            print(f"Error adjusting messages: {e}")
            continue
 
        # Chat Completion 요청 데이터 생성
        completion_request_data = {
            "messages": adjusted_messages,
            "maxTokens": 100,  # Chat Completion에서 사용할 토큰 수
            "temperature": 0.5,
            "topK": 0,
            "topP": 0.8,
            "repeatPenalty": 1.2,
            "stopBefore": [],
            "includeAiFilters": True,
            "seed": 0
        }
 
        try:
            response = completion_executor.execute(completion_request_data, stream=stream)
            if stream:
                response_text = parse_stream_response(response)
            else:
                response_text = parse_non_stream_response(response)
             
            messages.append({"role": "assistant", "content": response_text})
 
            # 대화 내역 표시
            print("\nAdjusted Messages:", adjusted_messages, "\n")
            print("System Prompt:", system_prompt)
            print("USER Input:", user_input)
            print("CLOVA Response:", response_text, "\n")
 
        except Exception as e:
            print(f"Error: {e}")

### Embedding API
+ Curl or Python for basic uses


In [5]:
import http.client
import json
import ssl

class HTTPSConnectionIgnoreSSL(http.client.HTTPSConnection):
    def __init__(self, *args, **kwargs):
        context = ssl._create_unverified_context()
        super().__init__(*args, context=context, **kwargs)

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = HTTPSConnectionIgnoreSSL(self._host)
        conn.request('POST', '/testapp/v1/api-tools/embedding/clir-emb-dolphin/2832d969b19148bdb52510c0cbf4c283', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['embedding']
        else:
            return 'Error'

# sample json file setup
sample = """{"text":"This is sample"}"""
print(sample)

# executor construction
completion_executor = CompletionExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY6VmI8/iRIe4ZndtlgZOQ60RQUeR4i9QAR2wMj5FUepl',
    api_key_primary_val = 'AQVDvkLRcV9fk8gwKYVlzEm4YLyfnb1rPrw7kyrU',
    request_id='93dbc51430f6463d938a82670c281962'
)

request_data = json.loads(sample, strict=False)

response_text = completion_executor.execute(request_data)
print(request_data)
print(response_text)


{"text":"This is sample"}
{'text': 'This is sample'}
[-0.5629949, -0.5761197, -0.54354006, 1.7467072, -1.1004752, -0.9674218, -0.2853109, 1.4539527, 0.70720434, 1.1020917, -0.74263704, 1.2994303, 1.20098, 1.2977431, -0.090564914, 0.22712049, -0.830937, -0.31083155, 0.78160745, 1.4026935, 0.04953583, -0.57326746, 1.2367895, 1.0083611, 1.1842182, -1.3119484, -1.0733888, -0.60352176, 0.8989068, 1.1703836, 0.22572279, 1.1384507, -1.219131, -0.41279534, 1.4092188, 0.45903075, -0.99971414, -1.2118713, -0.42943674, -0.33045897, -1.1080099, -1.4334337, -0.2332894, -0.51971513, 0.029274063, 1.1456181, -1.4166014, -1.4018109, -0.12772232, -1.0241814, 0.7093524, 0.20597534, -0.28263634, -0.33659467, 0.70287484, -1.369615, 1.103818, 0.7893418, -1.4935117, -0.9555506, 1.7648977, 1.3093429, -0.4834284, 0.52282965, 0.5125867, -0.6506602, 1.076987, -0.9533914, 0.41178623, -0.9787497, 0.6558708, -0.95989645, -1.2504759, -0.11618062, -0.708813, -1.1164984, -1.7315743, 0.7970522, 0.3945115, 1.1906227, 1.

### Querying

### Llamaparse

In [29]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

In [31]:
parser = LlamaParse(result_type="text")
file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader(
    input_dir="issuenote",
    recursive=True,
    file_extractor=file_extractor
)
docs = reader.load_data()

ValidationError: 1 validation error for LlamaParse
api_key
  Value error, The API key is required. [type=value_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

### Summarisation

In [20]:
len(documents[0].text)

1471

In [21]:
sample_dict = """{
  "texts" : [ "input text 1", "input text 2" ],
  "segMinSize" : 300,
  "includeAiFilters" : false,
  "autoSentenceSplitter" : true,
  "segCount" : -1,
  "segMaxSize" : 1000
}"""

# Convert dictionary to JSON string
sample_json = json.loads(sample_dict, strict=False)

print(sample_json)

{'texts': ['input text 1', 'input text 2'], 'segMinSize': 300, 'includeAiFilters': False, 'autoSentenceSplitter': True, 'segCount': -1, 'segMaxSize': 1000}


In [22]:
import json

# Use f-string formatting to insert `documents[0].text`
sample_dict = f"""{{
  "texts" : [ "{documents[0].text}", "{documents[1].text}" ],
  "segMinSize" : 300,
  "includeAiFilters" : false,
  "autoSentenceSplitter" : true,
  "segCount" : -1,
  "segMaxSize" : 1000
}}"""

# Load the JSON string into a dictionary
sample_json = json.loads(sample_dict, strict=False)

print(sample_json)

{'texts': ['1\n토큰증권(Security Token Offering)을 통한 녹색채권 발행 사례 및 시사점1    탄소중립 목표 달성을 위해서는 저탄소·친환경 분야에 막대한 자금이 소요될 것으로 예상되나, \n동 자금을 조달하기 위한 국내 녹색금융 시장의 성장세는 더딘 상황이다. 본 고는 우리나라의 녹\n색금융 활성화 방안으로 홍콩 등 해외 토큰형 녹색채권의 발행 사례를 살펴보고 국내 활용 가능성 \n등을 점검하였다.\n2    녹색채권은 친환경 프로젝트에 소요되는 자금을 조달하기 위해 발행되는 채권으로 자금의 사용처\n와 환경개선 효과에 대한 투명성이 매우 중요하다. 이에 따라 일반채권에 비해 엄격한 사전·사\n후 보고 및 외부 검증 절차를 두고 있다. 그러나 이처럼 복잡한 발행·보고 절차와 발행자와 투자\n자 간 구조적인 정보의 비대칭성(그린워싱 우려) 등은 녹색채권 시장 활성화의 주된 제약요인으로 \n꼽힌다.\n3    녹색채권을 블록체인을 기반으로 하는 토큰증권으로 발행하는 경우 조달자금의 사용처 및 환경영\n향에 대한 투명성 제고, 발행 및 사후보고 절차의 간소화, 투자자 기반 확대 등 다양한 효과가 기\n대된다. 이를 통해 국내기업, 특히 녹색금융 접근성이 낮았던 중소·중견기업에도 친환경 자금 \n조달 기회가 확대될 것으로 기대된다.\n4    해외 사례를 보면 홍콩에서는 당국 주도로 녹색국채를 토큰증권 형태로 시범 발행하여 규제 관련 \n불확실성을 해소하는 한편 스마트 계약 등을 통한 발행절차의 개선 효과를 검증하였다. 일본에서\n는 기업 및 금융기관이 협업을 통해 블록체인 및 IoT 기술을 활용한 토큰형 녹색채권을 발행하여, \n동 자금 사용처에 대한 투명성을 제고하고 환경영향 정보 수집 절차를 자동화하였다.\n5    우리나라도 녹색금융 활성화 방안의 일환으로 토큰증권의 활용 가능성을 검토할 필요가 있다. 특\n히 홍콩의 사례를 참고하여 초기에는 공공부문 등이 시범 발행을 통해 시장을 선도하는 방안을 고\n려해 볼 수 있다. 다만, 

In [144]:
# -*- coding: utf-8 -*-

import http.client
import json

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/summarization/v2/1b2c67d9a2c54841a59b74217426ff4e', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['text']
        else:
            return 'Error'


if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY6VmI8/iRIe4ZndtlgZOQ60RQUeR4i9QAR2wMj5FUepl',
        api_key_primary_val = 'AQVDvkLRcV9fk8gwKYVlzEm4YLyfnb1rPrw7kyrU',
        request_id='9d7090adc6874c1fa8dce15a3f32f5fd'
    )

    request_data = json.loads(sample_dict, strict=False)

    response_text = completion_executor.execute(request_data)
    print(request_data)
    print(response_text)


{'texts': ['1\n토큰증권(Security Token Offering)을 통한 녹색채권 발행 사례 및 시사점1    탄소중립 목표 달성을 위해서는 저탄소·친환경 분야에 막대한 자금이 소요될 것으로 예상되나, \n동 자금을 조달하기 위한 국내 녹색금융 시장의 성장세는 더딘 상황이다. 본 고는 우리나라의 녹\n색금융 활성화 방안으로 홍콩 등 해외 토큰형 녹색채권의 발행 사례를 살펴보고 국내 활용 가능성 \n등을 점검하였다.\n2    녹색채권은 친환경 프로젝트에 소요되는 자금을 조달하기 위해 발행되는 채권으로 자금의 사용처\n와 환경개선 효과에 대한 투명성이 매우 중요하다. 이에 따라 일반채권에 비해 엄격한 사전·사\n후 보고 및 외부 검증 절차를 두고 있다. 그러나 이처럼 복잡한 발행·보고 절차와 발행자와 투자\n자 간 구조적인 정보의 비대칭성(그린워싱 우려) 등은 녹색채권 시장 활성화의 주된 제약요인으로 \n꼽힌다.\n3    녹색채권을 블록체인을 기반으로 하는 토큰증권으로 발행하는 경우 조달자금의 사용처 및 환경영\n향에 대한 투명성 제고, 발행 및 사후보고 절차의 간소화, 투자자 기반 확대 등 다양한 효과가 기\n대된다. 이를 통해 국내기업, 특히 녹색금융 접근성이 낮았던 중소·중견기업에도 친환경 자금 \n조달 기회가 확대될 것으로 기대된다.\n4    해외 사례를 보면 홍콩에서는 당국 주도로 녹색국채를 토큰증권 형태로 시범 발행하여 규제 관련 \n불확실성을 해소하는 한편 스마트 계약 등을 통한 발행절차의 개선 효과를 검증하였다. 일본에서\n는 기업 및 금융기관이 협업을 통해 블록체인 및 IoT 기술을 활용한 토큰형 녹색채권을 발행하여, \n동 자금 사용처에 대한 투명성을 제고하고 환경영향 정보 수집 절차를 자동화하였다.\n5    우리나라도 녹색금융 활성화 방안의 일환으로 토큰증권의 활용 가능성을 검토할 필요가 있다. 특\n히 홍콩의 사례를 참고하여 초기에는 공공부문 등이 시범 발행을 통해 시장을 선도하는 방안을 고\n려해 볼 수 있다. 다만, 

### Demo workflow
https://www.ncloud-forums.com/topic/307/

In [14]:
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests

from llama_index.readers.web import SimpleWebPageReader

ModuleNotFoundError: No module named 'langchain_community'

In [15]:
# Raw Data Collection (Web Scraping)
url_to_filename_map = {}

with open("./data/clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]

print(urls)
folder_path = "issuenote"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:

    filename = url.split("/")[-1] + ".html"
    print(f"Downloading file: {filename}")
    file_path = os.path.join(folder_path, filename)
    
    # Prepare the full wget command as a single string
    wget_command = f"wget -O {file_path} {url} --no-check-certificate"
    
    # Print the full wget command for debugging
    print("Running command:", wget_command)
    
    try:
        # Execute the wget command using shell=True to match the printed string
        subprocess.run(wget_command, shell=True, check=True)
        url_to_filename_map[url] = filename
    except subprocess.CalledProcessError as e:
        print(f"Failed to download {url}: {e}")

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)


['https://www.ncloud-forums.com/topic/307']
Running command: wget -O issuenote/307.html https://www.ncloud-forums.com/topic/307 --no-check-certificate


--2024-10-30 09:52:47--  https://www.ncloud-forums.com/topic/307
Resolving www.ncloud-forums.com (www.ncloud-forums.com)... 211.249.59.16, 49.236.142.38
Connecting to www.ncloud-forums.com (www.ncloud-forums.com)|211.249.59.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658619 (643K) [text/html]
Saving to: ‘issuenote/307.html’

     0K .......... .......... .......... .......... ..........  7% 2.03M 0s
    50K .......... .......... .......... .......... .......... 15% 3.50M 0s
   100K .......... .......... .......... .......... .......... 23% 3.75M 0s
   150K .......... .......... .......... .......... .......... 31% 4.98M 0s
   200K .......... .......... .......... .......... .......... 38% 12.5M 0s
   250K .......... .......... .......... .......... .......... 46% 8.29M 0s
   300K .......... .......... .......... .......... .......... 54% 9.20M 0s
   350K .......... .......... .......... .......... .......... 62% 6.55M 0s
   400K .......... .......... ..